In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# from pprint import pprint
from lxml import html
# from lxml.html import fromstring
# import urllib.request
# from urllib.request import urlopen
import random
import re
# import scrapy
import datetime as dt
from selenium import webdriver
from tqdm import tqdm
import time

# DataFrame con todos los topics

In [2]:
url = 'https://www.filmaffinity.com/es/topics.php'

In [3]:
resp = requests.get(url)

In [4]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [5]:
topics = pd.DataFrame()
tname = []
tid = []
tnr = []
driver = webdriver.Firefox()
driver.get(url)
topics = driver.find_elements_by_class_name('topic')
for i in topics:
    try:
              
        tname.append(re.sub(' \(\d*\)','',i.text)) # topic name
        tid.append(re.findall('topic=(\d*)', i.get_attribute('href'))[0]) # topic id
        tnr.append(re.findall('\((\d*)\)', i.text)[-1]) # quantity of movies

        
    except:
        break
driver.quit()

In [6]:
topics = pd.DataFrame(index = tid, data = {'name': tname, 'qty': tnr})

In [7]:
for i,c in topics.iterrows():
    if c['qty'] != '':
        topics.loc[i, 'pages'] = int((int(c['qty'])-(int(c['qty'])%50))/50+1)
    else:
        topics.loc[i, 'pages'] = 0

In [8]:
topics['pages'] = topics['pages'].astype('int')

In [9]:
topics.head(5)

,name,qty,pages
928351,11-S,79,2
461156,3-D,792,16
836459,Abusos sexuales,529,11
791958,Acoso escolar/bullying,198,4
746104,Adolescencia,3356,68


# Magnitud de los datos a extraer

## Número de temas por los que iterar

In [10]:
topic_list = [x.text for x in soup.findAll('a', attrs={'class':'topic'})]
len(topic_list)

394

## Número de pelis a extraer

In [11]:
x = 0
for a in topic_list:
    try:
        x += int(re.findall('\((\d*)\)',a)[0])
    except:
        pass
x

310176

## Número de requests necesarios

In [12]:
topics['pages'].sum()

6403

# Ejecución

Para obtener más datos, habría que hacer un request para cada una de las películas para entrar en la web de cada una de ellas y extraer toda la información de su ficha. Serían más de 300.000 requests, por lo que sería necesario utilizar VPN o Tor.

Si se ejecuta este programa, sacará algo más de 20.000 películas y después dará error con status_code = 429

In [ ]:
for i,c in topics.iterrows():
    
    # extraer número de páginas de cada tema del df anterior
    pages = topics.loc[i, 'pages']
    
    # for p in páginas, iteraremos en los parámetros con el índice (i) y el nº página (p)
    for p in tqdm(range(1, pages+1)):
        
        # requests get beautiful soup 
        resp = requests.get(urltopic, params={'topic': f'{i}', 'attr': 'all', 'p': f'{p}'})
        soup = BeautifulSoup(resp.text, 'html.parser')
        pelis = soup.findAll('div', attrs={'class': 'record'})
        
                    
        for x in pelis:
            try: # Tengo que hacer try/except porque hay excepciones contadas con algún campo vacío
                
                alldb.append({'Título': x.find('div', attrs={'class': 'mc-title'}).find('a').text,
                   'Año': x.find('div', attrs={'class': 'mc-title'}).text.replace(' ','')[-5:-1],
                   'Director/es': x.find('div', attrs={'class': 'mc-director'}).text.replace(' (Creador)','').split(', '),
                   'Reparto': x.find('div', attrs={'class': 'mc-cast'}).text.split(', '),
                   'Género/s': x.find('div', attrs={'class': 'related-topics'}).text.split('\n')[1:-1],
                   'País': x.find('img', attrs={'class': 'nflag'}).get_attribute_list('title')[0],
                   'Link': re.findall('\/film(\d*)', x.find('div', attrs={'class': 'mc-title'}).find('a').get('href'))[0],
                    })
            except:
                pass
            
        if resp.status_code != 200:
            print(f'Status code: {resp.status_code}')
            print(f'Index: {i}')
            print(f'Page: {p}')
            driver = webdriver.Firefox()
            driver.get('https://www.filmaffinity.com/es/main.html')
            box = driver.find_element_by_id('recaptcha-anchor')
            box.click()
            driver.implicitly_wait(2)
            break
        
        #time.sleep(0.5)
        pass
        